In [155]:
obs = "xyxzzxyxyy"
e_var = ['x','y','z']
states = ['A','B']
d_trans = {'A':{'A':0.641, 'B':0.359},'B': {'A':0.729, 'B':0.271} }
d_emission = {'A':{'x':0.117, 'y':0.691, 'z':0.192},'B': {'x':0.097, 'y':0.42, 'z':0.483} }


In [156]:
import numpy as np

In [157]:
def prob_calc(obs, states, d_trans, d_emission):
	dict = { s:[] for s in states} 
	
	probabilities = {}
	for s in states:
		probabilities[s] = d_emission[s][obs[0]]
	
	for i in range(1, len(obs)):
		last_prob = probabilities
		probabilities = {}
		for curr_state in states:
			combinations = ((last_prob[last_state]*d_trans[last_state][curr_state]*d_emission[curr_state][obs[i]], last_state) for last_state in states)
			max_prob, last_state = max(combinations)
			probabilities[curr_state] = max_prob
			dict[curr_state].append(last_state)

	return dict,probabilities

def Viterbi(dict, probabilities, states):
	max_prob = -1
	max_dict = None
	for s in states:
		dict[s].append(s)
		if probabilities[s] > max_prob:
			max_dict = dict[s]
			max_prob = probabilities[s]
	max_dict = ''.join(max_dict)
	return max_dict

dict, probabilities = prob_calc(obs, states,  d_trans, d_emission)
path = Viterbi(dict,probabilities, states)
path

'AAABBAAAAA'

In [158]:
dict, probabilities

({'A': ['A', 'A', 'A', 'B', 'B', 'A', 'A', 'A', 'A', 'A'],
  'B': ['A', 'A', 'A', 'B', 'A', 'A', 'A', 'A', 'A', 'B']},
 {'A': 4.90330990435078e-08, 'B': 1.6691562960777426e-08})

In [ ]:
def estimateParameters( xList, pathList, transition, emission):
    transition = np.zeros_like(transition, dtype = float)
    emission = np.zeros_like(emission, dtype = float)
    n, l = emission.shape
    for i in range(len(pathList)-1):
        transition[pathList[i], pathList[i+1]] += 1
    for i in range(len(xList)):
        emission[pathList[i], xList[i]] += 1
    for i in range(n):
        sum1 = sum(transition[i,:])
        if 0 == sum1:
            transition[i,:] += 1/n
        else:
            transition[i,:] /= sum1
        sum2 = sum(emission[i,:])
        if 0 == sum2:
            emission[i,:] += 1/l
        else:
            emission[i,:] /= sum2
    return transition, emission

estimateParameters(obs,path,d_trans,d_emission)

In [ ]:
def softDecode( x, transition, emission, e_var, states):
    n = len(x)
    l = transition.shape[0]
    x2ind = {e_var[i]:i for i in range(len(e_var))}
    xList = [x2ind[x[i]] for i in range(len(x))]
    p = [[0 for _ in range(l)] for __ in range(n)]
    q = [[0 for _ in range(l)] for __ in range(n)]
    for k in range(l):
        p[0][k] = emission[k, xList[0]]/l
    for i in range(1, n):
        for k in range(l):
            p[i][k] = sum([p[i-1][z]*transition[z, k]*emission[k, xList[i]] for z in range(l)])
    fsink = sum(p[n-1])

    for k in range(l):
        q[n-1][k] = 1
    for i in range(n-2, -1, -1):
        for k in range(l):
            q[i][k] = sum([q[i+1][z]*transition[k, z]*emission[z, xList[i+1]] for z in range(l)])
    
    d = np.zeros((n, l), dtype = float)
    for i in range(n):
        for k in range(l):
            d[i, k] = p[i][k]*q[i][k]/fsink

    return d

In [159]:
f = '''100
--------
xxxzyzzxxzxyzxzxyxxzyzyzyyyyzzxxxzzxzyzzzxyxzzzxyzzxxxxzzzxyyxzzzzzyzzzxxzzxxxyxyzzyxzxxxyxzyxxyzyxz
--------
x y z
--------
A B
--------
A B
A 0.582 0.418
B 0.272 0.728
--------
x y z
A 0.129 0.35 0.52
B 0.422 0.151 0.426
'''

data = f.strip().split()
data

['100',
 '--------',
 'xxxzyzzxxzxyzxzxyxxzyzyzyyyyzzxxxzzxzyzzzxyxzzzxyzzxxxxzzzxyyxzzzzzyzzzxxzzxxxyxyzzyxzxxxyxzyxxyzyxz',
 '--------',
 'x',
 'y',
 'z',
 '--------',
 'A',
 'B',
 '--------',
 'A',
 'B',
 'A',
 '0.582',
 '0.418',
 'B',
 '0.272',
 '0.728',
 '--------',
 'x',
 'y',
 'z',
 'A',
 '0.129',
 '0.35',
 '0.52',
 'B',
 '0.422',
 '0.151',
 '0.426']

In [160]:
iterNo = int(data[0])
x = data[2]
ind = [i for i in range(len(data)) if '--------' == data[i]]
alphabet = data[ind[1]+1:ind[2]]
states = data[ind[2]+1:ind[3]]
transition = np.zeros((len(states), len(states)), dtype = float)
emission = np.zeros((len(states), len(alphabet)), dtype = float)
print(x,alphabet,emission,transition,states,ind)

xxxzyzzxxzxyzxzxyxxzyzyzyyyyzzxxxzzxzyzzzxyxzzzxyzzxxxxzzzxyyxzzzzzyzzzxxzzxxxyxyzzyxzxxxyxzyxxyzyxz ['x', 'y', 'z'] [[0. 0. 0.]
 [0. 0. 0.]] [[0. 0.]
 [0. 0.]] ['A', 'B'] [1, 3, 7, 10, 19]


In [161]:
for i in range(len(states)):
    transition[i, :] = [float(d) for d in data[ind[3]+len(states)+2+i*(len(states)+1):ind[3]+len(states)+1+(i+1)*(len(states)+1)]]
    emission[i, :] = [float(d) for d in data[ind[4]+len(alphabet)+2+i*(len(alphabet)+1):ind[4]+len(alphabet)+1+(i+1)*(len(alphabet)+1)]]
print(emission)
print(transition)

[[0.129 0.35  0.52 ]
 [0.422 0.151 0.426]]
[[0.582 0.418]
 [0.272 0.728]]


In [162]:
def decode(xList, transition, emission):
    n, l = len(xList), transition.shape[0]
    score = [[1. for i in range(l)] for i in range(n)]
    backtrack = [[None for i in range(l)] for i in range(n)]
    for k in range(l):
        score[0][k] = emission[k, xList[0]]/l
    #print(score)
    for i in range(1, n):
        for k in range(l):
            currScore = [score[i-1][kpre] * transition[kpre, k] * emission[k, xList[i]] for kpre in range(l)]
            ind = np.argmax(currScore)
            backtrack[i][k] = ind
            score[i][k] = currScore[ind]
    currState = np.argmax(score[n-1])
    pathList = [currState]
    for i in range(n-1, 0, -1):
        currState = backtrack[i][currState]
        pathList.insert(0, currState)
    return pathList

x2ind, ind2x = {alphabet[i]:i for i in range(len(alphabet))}, {i:alphabet[i] for i in range(len(alphabet))}
path2ind, ind2path = {states[i]:i for i in range(len(states))}, {i:states[i] for i in range(len(states))}
xList = [x2ind[x[i]] for i in range(len(x))]
pathList = decode(xList, transition, emission)

In [163]:
def estimateParameters(xList,pathList,transition,emission):
    transition = np.zeros_like(transition, dtype = float)
    emission = np.zeros_like(emission, dtype = float)
    n, l = emission.shape
    for i in range(len(pathList)-1):
        transition[pathList[i], pathList[i+1]] += 1
    for i in range(len(xList)):
        emission[pathList[i], xList[i]] += 1
    for i in range(n):
        sum1 = sum(transition[i,:])
        if 0 == sum1:
            transition[i,:] += 1/n
        else:
            transition[i,:] /= sum1
        sum2 = sum(emission[i,:])
        if 0 == sum2:
            emission[i,:] += 1/l
        else:
            emission[i,:] /= sum2
    return transition, emission
transition, emission = estimateParameters(xList,pathList,transition,emission)
transition, emission#, pathList

(array([[0.875     , 0.125     ],
        [0.01098901, 0.98901099]]),
 array([[0.        , 0.75      , 0.25      ],
        [0.40217391, 0.17391304, 0.42391304]]))

In [164]:
def softDecode(x, transition, emission, alphabet, states):
    n = len(x)
    l = transition.shape[0]
    x2ind = {alphabet[i]:i for i in range(len(alphabet))}
    xList = [x2ind[x[i]] for i in range(len(x))]
    forward = [[0 for _ in range(l)] for __ in range(n)]
    backward = [[0 for _ in range(l)] for __ in range(n)]
    for k in range(l):
        forward[0][k] = emission[k, xList[0]]/l
    for i in range(1, n):
        for k in range(l):
            forward[i][k] = sum([forward[i-1][kpre]*transition[kpre, k]*emission[k, xList[i]] for kpre in range(l)])
    fsink = sum(forward[n-1])

    for k in range(l):
        backward[n-1][k] = 1
    for i in range(n-2, -1, -1):
        for k in range(l):
            backward[i][k] = sum([backward[i+1][kpre]*transition[k, kpre]*emission[kpre, xList[i+1]] for kpre in range(l)])
    
    Pr = np.zeros((n, l), dtype = float)
    for i in range(n):
        for k in range(l):
            Pr[i, k] = forward[i][k]*backward[i][k]/fsink

    Pr2 = np.zeros((l, l, n-1), dtype = float)
    for k1 in range(l):
        for k2 in range(l):
            for i in range(n-1):
                Pr2[k1, k2, i] = forward[i][k1]*transition[k1, k2]*emission[k2, xList[i+1]]*\
                backward[i+1][k2]/fsink

    return Pr, Pr2

p1, p2 = softDecode(x, transition, emission, alphabet, states)
print(p1)
print("--------")
print(p2)

[[0.00000000e+00 1.00000000e+00]
 [0.00000000e+00 1.00000000e+00]
 [0.00000000e+00 1.00000000e+00]
 [6.37717472e-03 9.93622825e-01]
 [1.62252889e-02 9.83774711e-01]
 [8.41501011e-03 9.91584990e-01]
 [3.70575998e-03 9.96294240e-01]
 [0.00000000e+00 1.00000000e+00]
 [0.00000000e+00 1.00000000e+00]
 [8.27503970e-04 9.99172496e-01]
 [0.00000000e+00 1.00000000e+00]
 [9.12433206e-03 9.90875668e-01]
 [3.94837528e-03 9.96051625e-01]
 [0.00000000e+00 1.00000000e+00]
 [8.27503970e-04 9.99172496e-01]
 [0.00000000e+00 1.00000000e+00]
 [6.01967828e-03 9.93980322e-01]
 [0.00000000e+00 1.00000000e+00]
 [0.00000000e+00 1.00000000e+00]
 [1.69634848e-01 8.30365152e-01]
 [4.93533409e-01 5.06466591e-01]
 [5.74050421e-01 4.25949579e-01]
 [7.35421053e-01 2.64578947e-01]
 [7.71948123e-01 2.28051877e-01]
 [8.53406140e-01 1.46593860e-01]
 [8.67839647e-01 1.32160353e-01]
 [8.45373178e-01 1.54626822e-01]
 [7.39116408e-01 2.60883592e-01]
 [3.27298192e-01 6.72701808e-01]
 [1.12776031e-01 8.87223969e-01]
 [0.000000

In [184]:
f = '''
10
--------
xzyyzyzyxy
--------
x y z
--------
A B
--------
A B
A 0.019 0.981
B 0.668 0.332
--------
x y z
A 0.175 0.003 0.821
B 0.196 0.512 0.293

 '''

data = f.strip().split()
iterNo = int(data[0])
x = data[2]
ind = [i for i in range(len(data)) if '--------' == data[i]]
alphabet = data[ind[1]+1:ind[2]]
states = data[ind[2]+1:ind[3]]
transition = np.zeros((len(states), len(states)), dtype = float)
emission = np.zeros((len(states), len(alphabet)), dtype = float)
for i in range(len(states)):
    transition[i, :] = [float(d) for d in data[ind[3]+len(states)+2+i*(len(states)+1):ind[3]+len(states)+1+(i+1)*(len(states)+1)]]
    emission[i, :] = [float(d) for d in data[ind[4]+len(alphabet)+2+i*(len(alphabet)+1):ind[4]+len(alphabet)+1+(i+1)*(len(alphabet)+1)]]
#print(emission)
#print(transition)
print(x,alphabet,emission,transition,states,ind)


xzyyzyzyxy ['x', 'y', 'z'] [[0.175 0.003 0.821]
 [0.196 0.512 0.293]] [[0.019 0.981]
 [0.668 0.332]] ['A', 'B'] [1, 3, 7, 10, 19]


In [185]:
l = transition.shape[0]
x2ind = {alphabet[i]:i for i in range(len(alphabet))}
xList = [x2ind[x[i]] for i in range(len(x))]
#prob_calc(obs, states, d_trans, d_emission)
#pathList = decode(xList, transition, emission)
print(xList,pathList)


[0, 2, 1, 1, 2, 1, 2, 1, 0, 1] [1, 0, 1, 1, 0, 1, 0, 1, 0, 1]


In [189]:

def softDecode1(xList, transition, emission, alphabet):
    n = len(xList)
    l = transition.shape[0]
    forward = [[0 for _ in range(l)] for __ in range(n)]
    backward = [[0 for _ in range(l)] for __ in range(n)]
    for k in range(l):
        forward[0][k] = emission[k, xList[0]]/l
    for i in range(1, n):
        for k in range(l):
            forward[i][k] = sum([forward[i-1][kpre]*transition[kpre, k]*emission[k, xList[i]] for kpre in range(l)])
    fsink = sum(forward[n-1])

    for k in range(l):
        backward[n-1][k] = 1
    for i in range(n-2, -1, -1):
        for k in range(l):
            backward[i][k] = sum([backward[i+1][kpre]*transition[k, kpre]*emission[kpre, xList[i+1]] for kpre in range(l)])
    
    Pr = np.zeros((l, n), dtype = float)
    for i in range(n):
        for k in range(l):
            Pr[k, i] = forward[i][k]*backward[i][k]/fsink
    
    Pr2 = np.zeros((l, l, n-1), dtype = float)
    for k1 in range(l):
        for k2 in range(l):
            for i in range(n-1):
                Pr2[k1, k2, i] = forward[i][k1]*transition[k1, k2]*emission[k2, xList[i+1]]*\
                backward[i+1][k2]/fsink

    nAlphabet = len(alphabet)
    n = len(xList)
    l = Pr2.shape[0]
    #print(l,n)
    transition = np.zeros((l, l), dtype = float)
    emission = np.zeros((l, nAlphabet), dtype = float)
    for k1 in range(l):
        for k2 in range(l):
            transition[k1, k2] = sum(Pr2[k1, k2, :])
    #print(transition)
    for k in range(l):
        for i in range(n):
            emission[k, xList[i]] += Pr[k, i]

    for i in range(l):
        sum1 = sum(transition[i,:])
        if 0 == sum1:
            transition[i,:] += 1/l
        else:
            transition[i,:] /= sum1
        sum2 = sum(emission[i,:])
        if 0 == sum2:
            emission[i,:] += 1/nAlphabet
        else:
            emission[i,:] /= sum2
    return transition, emission


for _ in range(10):
    transition, emission = softDecode1(xList, transition, emission, alphabet)
transition, emission

(array([[3.96739268e-13, 1.00000000e+00],
        [7.85779011e-01, 2.14220989e-01]]),
 array([[2.42459188e-01, 1.44694390e-42, 7.57540812e-01],
        [1.72160393e-01, 8.27839607e-01, 6.77696200e-29]]))